In [6]:
#from UserItemData import UserItemData
from MovieData import MovieData
from RandomPredictor import RandomPredictor
from Recommender import Recommender
from AveragePredictor import AveragePredictor
from ViewsPredictor import ViewsPredictor
from STDPredictor import STDPredictor
#from ItemBasedPredictor import ItemBasedPredictor

# 1 Branje ocen
""" uim = UserItemData('data/user_ratedmovies.dat')
print(uim.nratings())
uim = UserItemData('data/user_ratedmovies.dat',  '12.1.2007', '16.2.2008', 100)
print(uim.nratings()) """

# 2 Branje filmov
""" md = MovieData('data/movies.dat')
print(md.get_title(1)) """

# 3 Naključni prediktor
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Film: {}, ocena: {}".format(md.get_title(item), pred[item])) """

# 4 Priporočanje
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """

# 5 Napovedovanje s povprečjem
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
avg_predictor = AveragePredictor(b=0)
avg_predictor.fit(uim)
predictions = avg_predictor.predict(78)
for movie_id, predicted_rating in predictions:
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), predicted_rating)) """

# 6 Priporočanje najbolj gledanih filmov
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
views_predictor = ViewsPredictor()
views_predictor.fit(uim)
predictions = views_predictor.predict()
for movie_id, view_count in predictions:
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), view_count)) """

# 7 Priporočanje kontroverznih filmov
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = STDPredictor(100)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """


""" import csv
from collections import defaultdict

def count_movies_rated_by_users(file_path):
    # Dictionary to store user ID and their corresponding count of rated movies
    user_movie_counts = defaultdict(int)
    
    # Open and read the file
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter='\t')  # Adjust delimiter if needed
        
        # Skip the header
        next(reader, None)
        
        # Process each row in the file
        for row in reader:
            user_id = int(row[0])  # Assuming first column is user ID
            user_movie_counts[user_id] += 1
    
    # Print the results
    for user_id, movie_count in user_movie_counts.items():
        if user_id==78:
            print(f"User {user_id} rated {movie_count} movies.")

#count_movies_rated_by_users('data/user_ratedmovies.dat') """

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
import numpy as np


from datetime import datetime

class UserItemData:
    def __init__(self, path, from_date=None, to_date=None, min_ratings=None):
        self.path = path
        self.from_date = datetime.strptime(from_date, '%d.%m.%Y') if from_date else None
        self.to_date = datetime.strptime(to_date, '%d.%m.%Y') if to_date else None
        self.min_ratings = min_ratings
        self.data = []
        self._read_data()

    def _read_data(self):
        with open(self.path, 'r', encoding='utf-8') as file:
            ratings_count = {}
            next(file)

            for line in file:
                fields = line.strip().split('\t')
                if len(fields) < 9:
                    print("input insufficient")
                    continue

                user_id = int(fields[0])
                movie_id = int(fields[1])
                rating = float(fields[2])

                date_time = datetime(
                    year=int(fields[5]),
                    month=int(fields[4]),
                    day=int(fields[3]),
                    hour=int(fields[6]),
                    minute=int(fields[7]),
                    second=int(fields[8])
                )
                #print(date_time)

                # conditions
                if self.from_date and date_time < self.from_date:
                    continue
                if self.to_date and date_time > self.to_date:
                    continue

                # movies, number of ratings
                ratings_count[movie_id] = ratings_count.get(movie_id, 0) + 1

                # conditions
                if self.min_ratings and ratings_count[movie_id] < self.min_ratings:
                    continue

                self.data.append((user_id, movie_id, rating, date_time))
                #if user_id == 42059:
                    #print((user_id, movie_id, rating, date_time))


from collections import defaultdict

class ItemBasedPredictor:
    def __init__(self, min_values=0, threshold=0):
        self.min_values = min_values
        self.threshold = threshold
        self.similarities = {}
        self.movie_ratings = {}
        self.all_fit_movies = []
        self.global_avg = 0;

    def fit(self, X):
        self.movie_ratings = defaultdict(dict)  # {movie_id: {user_id: rating}}
        for user_id, movie_id, rating, _ in X.data:
            #if user_id == 71534 :
                #print(user_id, movie_id)
            self.movie_ratings[movie_id][user_id] = rating
            # Loop over movies in the movie_ratings dictionary
            #for movie_id, ratings in self.movie_ratings.items():
                #if 71534 in ratings:  # Check if user 71534 has rated this movie
                    #print(movie_id)  # Print the movie_id of the rated movies

        
        all_ratings = [rating for movie_ratings in self.movie_ratings.values() for rating in movie_ratings.values()]
        self.global_avg = np.mean(all_ratings)
        
        movie_ids = list(self.movie_ratings.keys())
        self.similarities = defaultdict(dict)
        
        for i, movie1 in enumerate(movie_ids):
            for movie2 in movie_ids[i+1:]:
                sim = self.similarity(movie1, movie2)
                #print(sim)
                self.similarities[movie1][movie2] = sim
                self.similarities[movie2][movie1] = sim
                if movie1 not in self.all_fit_movies:
                    self.all_fit_movies.append(movie1)
                if movie2 not in self.all_fit_movies:
                    self.all_fit_movies.append(movie2)


    def similarity(self, p1, p2):
        # Pridobimo ocene obeh filmov
        users_p1 = self.movie_ratings[p1]
        users_p2 = self.movie_ratings[p2]
        
        # Seznam skupnih uporabnikov
        common_users = set(users_p1.keys()).intersection(users_p2.keys())
        if len(common_users) < self.min_values:
            print(f"Not enough common users between movies {p1} and {p2}")
            return 0  # If there are not enough common users

        # Pridobimo ocene za skupne uporabnike
        ratings_p1 = np.array([users_p1[user] for user in common_users])
        ratings_p2 = np.array([users_p2[user] for user in common_users])

        # Povprečne ocene uporabnikov
        avg_p1 = np.mean(ratings_p1)
        avg_p2 = np.mean(ratings_p2)

        # Odštejemo povprečja, da "centriramo" ocene
        centered_p1 = ratings_p1 - avg_p1
        centered_p2 = ratings_p2 - avg_p2

        # Izračunamo števnik (dot produkt)
        numerator = np.dot(centered_p1, centered_p2)
        #print(numerator)
        # Izračunamo imenovalec (norme)
        denominator = np.sqrt(np.sum(centered_p1**2)) * np.sqrt(np.sum(centered_p2**2))
        
        if denominator == 0:
            return 0

        # Izračunamo popravljeno cosinusno podobnost
        sim = numerator / denominator
        #print(sim)
        return sim #if sim >= self.threshold else 0


    def predict(self, user_id):

        # Dictionary to hold predicted ratings for each movie
        predicted_ratings = {}

        # Step 1: Get all movies rated by the user
        rated_movies = {
            movie_id for movie_id, stats in self.movie_ratings.items() if user_id in stats
        }

        # Step 2: Check which rated movies are in all_fit_movies
        relevant_movies = rated_movies.intersection(self.all_fit_movies)
        print(relevant_movies)
        # Step 3: Iterate over all movies in the movie_ratings dictionary
        for movie_id, stats in self.movie_ratings.items():
            # Skip movies that the user has already rated
            if movie_id in rated_movies:
                continue  # User has rated this movie
            
            # Initialize variables for weighted sum and similarity sum
            weighted_sum = 0  # To store the weighted sum of similarities
            similarity_sum = 0  # To store the sum of similarities for normalization

            # Iterate through relevant movies to calculate similarities
            for other_movie_id in relevant_movies:
                # Skip if there is no similarity data between movie_id and other_movie_id
                if movie_id not in self.similarities or other_movie_id not in self.similarities[movie_id]:
                    continue

                # Fetch similarity between the current movie and the other movie
                similarity = self.similarities[movie_id][other_movie_id]
                
                # Fetch the user's rating for the other movie
                other_movie_rating = self.movie_ratings[other_movie_id][user_id]
                
                # Add the weighted rating to the weighted_sum
                weighted_sum += similarity * (other_movie_rating - self.global_avg)
                
                # Add the absolute value of similarity to similarity_sum for normalization
                similarity_sum += abs(similarity)
            
            # If there is no similarity data, use global average
            if similarity_sum == 0:
                predicted_ratings[movie_id] = self.global_avg
            else:
                # Calculate predicted rating
                predicted_ratings[movie_id] = self.global_avg + (weighted_sum / similarity_sum)
        
        return predicted_ratings






    
    def predict_all(self, n=20):
        #return len(self.similarities.items())
        sorted_similarities = sorted(
            [(movie1, movie2, sim) for movie1, movie2_dict in self.similarities.items() for movie2, sim in movie2_dict.items()],
            key=lambda x: x[2], reverse=True
        )
        return sorted_similarities[:n]


# 1
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)

#2571 356
#Movie 2571 has similarities with: [356]
#Movie 356 has similarities with: [2571]
""" print("Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780): ", rp.similarity(1580, 780))
print("Predictions for 78: ") """
rec_items = rec.recommend(78  , n=15, rec_seen=False)
""" for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """

# 2
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor(min_values=1000)
rp.fit(uim)
top_similar_pairs = rp.get_top_most_similar_pairs(n=20)
print(top_similar_pairs)
for movie1, movie2, similarity in top_similar_pairs:
    print(f"Film1: {md.get_title(movie1)}, Film2: {md.get_title(movie2)}, podobnost: {similarity}")  """

set()


' md = MovieData(\'data/movies.dat\')\nuim = UserItemData(\'data/user_ratedmovies.dat\', min_ratings=1000)\nrp = ItemBasedPredictor(min_values=1000)\nrp.fit(uim)\ntop_similar_pairs = rp.get_top_most_similar_pairs(n=20)\nprint(top_similar_pairs)\nfor movie1, movie2, similarity in top_similar_pairs:\n    print(f"Film1: {md.get_title(movie1)}, Film2: {md.get_title(movie2)}, podobnost: {similarity}")  '